# The Project #
1. This is a project with minimal scaffolding. Expect to use the the discussion forums to gain insights! It’s not cheating to ask others for opinions or perspectives!
2. Be inquisitive, try out new things.
3. Use the previous modules for insights into how to complete the functions! You'll have to combine Pillow, OpenCV, and Pytesseract
4. There are hints provided in Coursera, feel free to explore the hints if needed. Each hint provide progressively more details on how to solve the issue. This project is intended to be comprehensive and difficult if you do it without the hints.

### The Assignment ###
Take a [ZIP file](https://en.wikipedia.org/wiki/Zip_(file_format)) of images and process them, using a [library built into python](https://docs.python.org/3/library/zipfile.html) that you need to learn how to use. A ZIP file takes several different files and compresses them, thus saving space, into one single file. The files in the ZIP file we provide are newspaper images (like you saw in week 3). Your task is to write python code which allows one to search through the images looking for the occurrences of keywords and faces. E.g. if you search for "pizza" it will return a contact sheet of all of the faces which were located on the newspaper page which mentions "pizza". This will test your ability to learn a new ([library](https://docs.python.org/3/library/zipfile.html)), your ability to use OpenCV to detect faces, your ability to use tesseract to do optical character recognition, and your ability to use PIL to composite images together into contact sheets.

Each page of the newspapers is saved as a single PNG image in a file called [images.zip](./readonly/images.zip). These newspapers are in english, and contain a variety of stories, advertisements and images. Note: This file is fairly large (~200 MB) and may take some time to work with, I would encourage you to use [small_img.zip](./readonly/small_img.zip) for testing.

Here's an example of the output expected. Using the [small_img.zip](./readonly/small_img.zip) file, if I search for the string "Christopher" I should see the following image:
![Christopher Search](./readonly/small_project.png)
If I were to use the [images.zip](./readonly/images.zip) file and search for "Mark" I should see the following image (note that there are times when there are no faces on a page, but a word is found!):
![Mark Search](./readonly/large_project.png)

Note: That big file can take some time to process - for me it took nearly ten minutes! Use the small one for testing.

In [9]:
from zipfile import ZipFile

from PIL import Image
from PIL import ImageDraw
from kraken import pageseg
import pytesseract 
import cv2 as cv
import numpy as np

# loading the face detection classifier
face_cascade = cv.CascadeClassifier('readonly/haarcascade_frontalface_default.xml')
# the rest is up to you!

# A global list of dictionary objects PIL_Image, bounding boxes, text
gl = []

#debug
#image = Image.open("readonly/text.png")
#text = pytesseract.image_to_string(image)
#print(text)
#image.close()

### Some useful functions

In [33]:
def face_detect(pil_img):
    '''
    takes an image, finds the faces on it (if any), and returns a list of coordinate-tuples that bound the faces found
    
    :param: a PIL.Image object
    
    :returns: a list, containing 4-tuples
    '''
    boxes = []
    
    # convert img from PIL format into opencv
    cv2_image_gray = cv.cvtColor(np.array(pil_img), cv.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(cv2_image_gray, scaleFactor =1.3, minNeighbors = 5).tolist()
    
    if (len(faces) > 0):
        for (x, y, w, h) in faces:
            boxes.append([x, y, x+w, y+h])
    
    return boxes

Go through the zip archive

In [29]:
small_file = 'readonly/small_img.zip'
big_file = 'readonly/images.zip'


def search(zipfile=small_file, word='Christopher'):
    zf = ZipFile(zipfile, mode='r')
    
    search_word = word.lower()

    for el in zf.infolist():
        with zf.open(el) as file:
            img = Image.open(file)
        
            # debug
            #print(img.size, img.mode, len(img.getdata()))
            #display(img)
            #print(file.name)
        
            # process OCR
            text = pytesseract.image_to_string(img).lower()
            print(text[:16])
       
            if (search_word in text):
                print("Found search_word {}".format(search_word))
                # create a dictionary object
                boxes = face_detect(img)
        
                new_entry = {"name":file.name, "image":img.copy(), "boxes":boxes, "text":text}
        
                gl.append(new_entry)
            


In [34]:
search()

 

ann arbor, mi
Found search_word christopher
2a — wednesday, 
the michigan dai
4a, 5a — wednesd
Found search_word christopher


In [35]:
print(gl[0]boxes)

AttributeError: 'dict' object has no attribute 'boxes'

In [16]:
for el in gl:
    print(el['name'])
        
    print("----------- OCR text -----------------------")
    #text = pytesseract.image_to_string(el["image"])
    print(el['text'][:128])
    
    # To test this, lets use display
    # display(show_boxes(el['image'].convert('1')))
    # break

In [3]:
zf.close()

a-0.png
----------- OCR text -----------------------
 

ann arbor, michigan

wednesday, november 5, 2014

che michigan daily

michigandaily.com

big day for republicans

snyder earn
a-1.png
----------- OCR text -----------------------
2a — wednesday, november 5, 2014

students vote, watch midterm election 2014

the michigan daily — michigandaily.com

 

che mic
a-2.png
----------- OCR text -----------------------
the michigan daily — michigandaily.com

page 3a — wednesday, november 5, 2014

 

che michigan daily

  

bic

oe

— # eit :
cae
a-3.png
----------- OCR text -----------------------
4a, 5a — wednesday, november 5, 2014

the michigan daily — michigandaily.com

 

 

luna anna archey/daily

ann arbor mayor elec
